In [ ]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! python -m spacy download ru_core_news_md

2022-09-03 07:24:10.298347: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
import ru_core_news_md
nlp = ru_core_news_md.load()

In [ ]:
data = pd.read_csv("test_data.csv")
print(data.shape)
data.head(5)

(480, 4)


,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [ ]:
data['dlg_id'].unique()

array([0, 1, 2, 3, 4, 5])

In [ ]:
data['role'].unique()

array(['client', 'manager'], dtype=object)

### Проведем предобработку данных

Очистим текст

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    return text


In [ ]:
data['text']=data['text'].apply(lambda x: clean_text(x), 1)
data.head(4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Possible nested set at position 39
  if __name__ == '__main__':


,dlg_id,line_n,role,text
0,0,0,client,алло
1,0,1,manager,алло здравствуйте
2,0,2,client,добрый день
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...


### Извлечем реплики с приветствием – где менеджер поздоровался

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)

In [ ]:
greeting = ['здравствуйте', 'здравствуй', 'добрый день', 'добрый вечер', 'доброе утро',
           'привет', 'приветствую вас', 'доброго дня', 'добрый', 'мое почтение', 'салют',
           'как поживаешь', 'добрейшего', 'рады приветствовать']

In [ ]:
patterns = [nlp(text) for text in greeting]  

In [ ]:
phrase_matcher.add('AI', None, *patterns)

In [ ]:
data['greeting']=data.loc[data['role'] == 'manager']['text'].apply(lambda x: nlp(x)[phrase_matcher(nlp(x))[0][1]:phrase_matcher(nlp(x))[0][2]].text if phrase_matcher(nlp(x)) else np.nan, 1)

In [ ]:
data.head(4)

,dlg_id,line_n,role,text,greeting
0,0,0,client,алло,NaN
1,0,1,manager,алло здравствуйте,здравствуйте
2,0,2,client,добрый день,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,NaN


### Извлечем реплики, где менеджер представил себя

In [ ]:
introduction = ['меня зовут', 'с вами говорит', 'это менеджер', 'зовут меня',
           'это ангелина', 'это анастасия', 'это максим', 'разрешите представится']

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns_introduction = [nlp(text) for text in introduction]
phrase_matcher.add('AII', None, *patterns_introduction)

In [ ]:
data['introduction'] = data.loc[data['role'] == 'manager']['text'].apply(lambda x: nlp(x)[phrase_matcher(nlp(x))[0][1]:phrase_matcher(nlp(x))[0][2]].text if phrase_matcher(nlp(x)) else np.nan, 1)
data.head(5)

,dlg_id,line_n,role,text,greeting,introduction
0,0,0,client,алло,NaN,NaN
1,0,1,manager,алло здравствуйте,здравствуйте,NaN
2,0,2,client,добрый день,NaN,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,NaN,меня зовут
4,0,4,client,ага,NaN,NaN


### Извлечем имя менеджера

In [ ]:
man_names = ['ангелина', 'анастасия', 'максим']

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns_name = [nlp(text) for text in man_names]
phrase_matcher.add('AIM', None, *patterns_name)  

In [ ]:
data['names'] = data.loc[data['role'] == 'manager']['text'].apply(lambda x: nlp(x)[phrase_matcher(nlp(x))[0][1]:phrase_matcher(nlp(x))[0][2]].text if phrase_matcher(nlp(x)) else np.nan, 1)
data.head(5)

,dlg_id,line_n,role,text,greeting,introduction,names
0,0,0,client,алло,NaN,NaN,NaN
1,0,1,manager,алло здравствуйте,здравствуйте,NaN,NaN
2,0,2,client,добрый день,NaN,NaN,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,NaN,меня зовут,ангелина
4,0,4,client,ага,NaN,NaN,NaN


### Извлечем название компании

In [ ]:
company = ['компания диджитал бизнес', 'диджитал бизнес', 'компания китобизнес']

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns_company = [nlp(text) for text in company]
phrase_matcher.add('AIC', None, *patterns_company)  

In [ ]:
data['company'] = data.loc[data['role'] == 'manager']['text'].apply(lambda x: nlp(x)[phrase_matcher(nlp(x))[0][1]:phrase_matcher(nlp(x))[0][2]].text if phrase_matcher(nlp(x)) else np.nan, 1)
data.head(5)

,dlg_id,line_n,role,text,greeting,introduction,names,company
0,0,0,client,алло,NaN,NaN,NaN,NaN
1,0,1,manager,алло здравствуйте,здравствуйте,NaN,NaN,NaN
2,0,2,client,добрый день,NaN,NaN,NaN,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,NaN,меня зовут,ангелина,компания диджитал бизнес
4,0,4,client,ага,NaN,NaN,NaN,NaN


### Извлечем реплики, где менеджер попрощался

In [ ]:
farewall = ['до свидания', 'пока', 'всего хорошего', 'хорошего дня', 'всего доброго', 'пока-пока', 'до встречи']

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
patterns_farewall = [nlp(text) for text in farewall]
phrase_matcher.add('AIM', None, *patterns_farewall)  

In [ ]:
data['farewall'] = data.loc[data['role'] == 'manager']['text'].apply(lambda x: nlp(x)[phrase_matcher(nlp(x))[0][1]:phrase_matcher(nlp(x))[0][2]].text if phrase_matcher(nlp(x)) else np.nan, 1)
data.head(5)

,dlg_id,line_n,role,text,greeting,introduction,names,company,farewall
0,0,0,client,алло,NaN,NaN,NaN,NaN,NaN
1,0,1,manager,алло здравствуйте,здравствуйте,NaN,NaN,NaN,NaN
2,0,2,client,добрый день,NaN,NaN,NaN,NaN,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,NaN,меня зовут,ангелина,компания диджитал бизнес,NaN
4,0,4,client,ага,NaN,NaN,NaN,NaN,NaN


### Проверим требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


In [84]:
check = data.groupby(['dlg_id'])['greeting'].apply(list).reset_index()
check = check.join(data.groupby(['dlg_id'])[''].apply(list))

In [86]:
check['greeting'] = check['greeting'].apply(lambda row:[x for x in row if str(x) != 'nan'])
check['farewall'] = check['farewall'].apply(lambda row:[x for x in row if str(x) != 'nan'])

In [87]:
check

,dlg_id,greeting,farewall
0,0,[здравствуйте],[всего хорошего]
1,1,[здравствуйте],"[всего хорошего, до свидания]"
2,2,[здравствуйте],[]
3,3,"[добрый, добрый]",[всего доброго]
4,4,[],[до свидания]
5,5,[],[до свидания]


In [115]:
check['comment'] = check.apply(lambda x: 'Менеджер выполнил требование' if ((len(x['greeting'])>0)&(len(x['farewall'])>0)) else 'Менеджер не выполнил требование',1)
check

,dlg_id,greeting,farewall,comment
0,0,[здравствуйте],[всего хорошего],Менеджер выполнил требование
1,1,[здравствуйте],"[всего хорошего, до свидания]",Менеджер выполнил требование
2,2,[здравствуйте],[],Менеджер не выполнил требование
3,3,"[добрый, добрый]",[всего доброго],Менеджер выполнил требование
4,4,[],[до свидания],Менеджер не выполнил требование
5,5,[],[до свидания],Менеджер не выполнил требование
